# Gradio UI & LLM Integration


# Add gradio in requirements.txt or pyproject.toml


In [24]:
## pyproject.toml

# [project]
# name = "llm-engineering"
# version = "0.1.0"
# requires-python = ">=3.11"
# dependencies = [
#     "gradio>=5.47.2",
#     # .. other dependencies
# ]

Import library

In [25]:
import os
from dotenv import load_dotenv
from openai import OpenAI

import gradio as gr

In [42]:
# Load .env variables securely
load_dotenv()

# Configuration variables from environment
TEST_API_KEY = os.getenv("TEST_API_KEY")
OLLAMA_API_BASE_URL = os.getenv("OLLAMA_BASE_URL") # e.g., http://localhost:11434/v1
LLM_MODEL = os.getenv("LLAMA3_3B") # The specific model to use

# Basic check to ensure required configuration is present
if (TEST_API_KEY or OLLAMA_API_BASE_URL or LLM_MODEL) is None:
    raise ValueError("One or more required environment variables is not set.")
else:
    print(f"Config loaded. \nBASE_URL: {OLLAMA_API_BASE_URL}, \nMODEL: {LLM_MODEL} \nAPI Key: {TEST_API_KEY[:10]}")

# Initialize the OpenAI client. base_url allows connection to local/non-OpenAI providers.
client = OpenAI(base_url=OLLAMA_API_BASE_URL, api_key=TEST_API_KEY)

Config loaded. 
BASE_URL: http://localhost:11434/v1, 
MODEL: llama3.2:3b 
API Key: DUMMY-API-


LLM API function to integrate with UI 

In [27]:
SYSTEM_PROMPT = """You are a helpful assistant that helps people find information.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
"""

# Initialize with user input
PAYLOAD = []
PAYLOAD.append({"role": "system", "content": SYSTEM_PROMPT})

In [37]:
# Take the request from user and call the LLM 
def chat(input: str) -> str:
    print(f"info: chat functions is called. \nInput: {input}")
    PAYLOAD.append({"role": "user", "content": input})
    response = client.chat.completions.create(model=LLM_MODEL,
                                              messages=PAYLOAD, stream=False)
    msg = response.choices[0].message.content
    PAYLOAD.append({"role": "assistant", "content": msg})

    print(f"Prompt: \n{PAYLOAD}")
    return msg

info: chat functions is called. 
Input: is my name Rahul ?


In [20]:
resp = chat("Hello, This is vivek from India. Tell me a funny fact.")

print(f"\nResponse: \n{resp}")

info: chat functions is called. 
Input: Hello, This is vivek from India. Tell me a funny fact.

Response: 
Okay, here’s a funny fact for you:

**Bananas are berries, but strawberries aren't.** 

It's a surprisingly complicated berry classification! 

---

Want to hear another one?


Create a Simple UI

In [ ]:

input_box = gr.Textbox(label="Ask me anything", placeholder="Type your question here...", type="text", lines=3, max_lines=10, max_length=100, autofocus=True)
output_box = gr.Textbox(label="Response", placeholder="You haven't asked anything yet...", type="text", lines=10, autoscroll=False, show_copy_button=True)

llm_chatting = gr.Interface(fn=chat, inputs=input_box, outputs=output_box, 
                    title="LLM Chat", 
                    description="ChatOllama", flagging_mode='never')

# Launch the Gradio interface
llm_chatting.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


info: chat functions is called. 
Input: hi my name is vivek I am s/w eng.
Prompt: 
[{'role': 'system', 'content': "You are a helpful assistant that helps people find information.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n"}, {'role': 'user', 'content': 'hi my name is vivek I am s/w eng.'}, {'role': 'assistant', 'content': "Hi Vivek! It's nice to meet you. Sorry, but I'm a large language model, I don't have personal interactions, so we can start fresh anytime.\n\nSo, you're a software engineer, right? What area of software engineering are you into (e.g., web development, mobile app development, AI/ML, etc.)? Or what's your current project or interest that you'd like to discuss?"}, {'role': 'user', 'content': 'what do you kno about me ?'}, {'role': 'assistant', 'content': "I don't know much about you personally. You introduced yourself as Vivek, a software engineer, but that's the extent of my knowledge so far.\n\nIf you feel comfortabl

Stop Server

In [38]:
llm_chatting.close()

# close all interfaces
gr.close_all()

Closing server running on port: 7860
